In [45]:
import pandas as pd
import numpy as np
import json
import csv
import string

In [24]:
df = pd.read_csv("MASTER_v3.csv")#, names=[i for i in range(num_col)])
df = df.iloc[1:,:]
df

/home/filippo/anaconda3/envs/HLP/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
1,adidas,carbonfootprint,allbirds,carbon,germany,tencel,polyester,shoes,adidasxallbirds,adidas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,adidas,emissions,carbonfootprint,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,adidas,ciie2021,lowcarbon,carbonfootprint,adidas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,adidas,carbonfootprint,saveearth,carbon,adidas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,adidas,carbonfootprint,sustainability,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57034,omen,round,green,rctid,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57035,omen,green,tech,cars,ev,electric,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57036,omen,nfc,nfc,gopackgo,green,superbowl,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57037,omen,eco,green,news,politics,supremecourt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
num_col = df.shape[1]
print(num_col)
rows = len(df)
print(rows)

38
57038


In [35]:
def columns_keeper(df):
    res_list = []
    for i in range(num_col):
            if str(df[str(i)].item()) != "nan":
                res_list.append(df[str(i)].item())
    return res_list

In [36]:
def get_couples(ls):
    copy_ls = ls.copy()
    couples = []
    for e1 in ls:
        copy_ls.pop(0)
        for e2 in copy_ls:
            couples.append([e1,e2])
    return couples

In [37]:
# Do couples of hashtags/brands for every tweet
couples_df = pd.DataFrame([], columns=['Node1','Node2'])
for i in range(rows):
    tweet_list = columns_keeper(df.iloc[[i]])
    tweet_list = list(dict.fromkeys(tweet_list))

    couples = get_couples(tweet_list)
    couples = pd.DataFrame(couples, columns=['Node1','Node2'])
    couples_df = pd.concat([couples_df, couples])

couples_df

,Node1,Node2
0,adidas,carbonfootprint
1,adidas,allbirds
2,adidas,carbon
3,adidas,germany
4,adidas,tencel
...,...,...
13,news,supremecourt
14,politics,supremecourt
0,omen,foofighters
1,omen,slowfashion


In [38]:
couples_df = couples_df.rename(columns={"Node1":"Source","Node2":"Target"})
couples_df

,Source,Target
0,adidas,carbonfootprint
1,adidas,allbirds
2,adidas,carbon
3,adidas,germany
4,adidas,tencel
...,...,...
13,news,supremecourt
14,politics,supremecourt
0,omen,foofighters
1,omen,slowfashion


In [39]:
# Count occurences to have a weighted list
df_weighted = couples_df.groupby(['Source','Target']).size().sort_values(ascending=False)
print(len(df_weighted))
weighted_list = []
for i in range(len(df_weighted)):
    node1 = df_weighted.index[i][0]
    node2 = df_weighted.index[i][1]
    count = df_weighted[i]
    weighted_list.append([node1,node2,count])
    
weighted_list = pd.DataFrame(weighted_list, columns=['Source', 'Target', 'Weigth'])
weighted_list   

638906


,Source,Target,Weigth
0,nike,flyknit,17925
1,nike,green,9395
2,adidas,green,3713
3,vans,green,2570
4,nike,sustainability,2397
...,...,...,...
638901,nikesportswear,vapormaxflyknit,1
638902,nikesportswear,volt,1
638903,nikesportswear,vscocam,1
638904,nikesportswear,vscogood,1


In [ ]:
weighted_list.to_csv("Weighted_edges.csv")

In [43]:
# Degree from you want to save
k = 1

In [44]:
w_list = weighted_list[weighted_list["Weigth"] >= k]
w_list = w_list.rename(columns={"Node1":"Source","Node2":"Target"})
w_list

,Source,Target,Weigth
0,nike,flyknit,17925
1,nike,green,9395
2,adidas,green,3713
3,vans,green,2570
4,nike,sustainability,2397
...,...,...,...
638901,nikesportswear,vapormaxflyknit,1
638902,nikesportswear,volt,1
638903,nikesportswear,vscocam,1
638904,nikesportswear,vscogood,1


In [46]:
# Remove strange character
printable = set(string.printable)
w_list["Source"] = w_list["Source"].apply(lambda row: ''.join(filter(lambda x: x in printable, row)))
w_list["Target"] = w_list["Target"].apply(lambda row: ''.join(filter(lambda x: x in printable, row)))
w_list

,Source,Target,Weigth
0,nike,flyknit,17925
1,nike,green,9395
2,adidas,green,3713
3,vans,green,2570
4,nike,sustainability,2397
...,...,...,...
638901,nikesportswear,vapormaxflyknit,1
638902,nikesportswear,volt,1
638903,nikesportswear,vscocam,1
638904,nikesportswear,vscogood,1


In [47]:
# Remove words with length less than 2
count = 0
new_w = []
for i, row in w_list.iterrows():
    if str(row["Source"]) != '' and str(row["Target"]) != '' and len(str(row["Source"])) >= 3 and len(str(row["Target"])) >= 3:
        count +=1
        new_w.append(row)
        
print(count)
w_list = pd.DataFrame(new_w, columns=["Source","Target","Weigth"])
w_list

608995


,Source,Target,Weigth
0,nike,flyknit,17925
1,nike,green,9395
2,adidas,green,3713
3,vans,green,2570
4,nike,sustainability,2397
...,...,...,...
638901,nikesportswear,vapormaxflyknit,1
638902,nikesportswear,volt,1
638903,nikesportswear,vscocam,1
638904,nikesportswear,vscogood,1


In [48]:
edgename = "edgelist_k" + str(k) +".csv"
w_list.to_csv(edgename, index=None)

In [49]:
# Get unique nodes
nodes1 = w_list["Source"].unique()
nodes2 = w_list["Target"].unique()
nodes = np.concatenate((nodes1,nodes2), axis=None)
nodes = np.unique(nodes)
print(len(nodes))

45461


In [50]:
nodes_csv = pd.DataFrame(nodes, columns=["Id"])
nodes_csv["Label"] = nodes
nodes_csv

,Id,Label
0,00s,00s
1,00sfashion,00sfashion
2,00sle9dcembre,00sle9dcembre
3,06jul,06jul
4,072100thermalinsulation,072100thermalinsulation
...,...,...
45456,zxflux,zxflux
45457,zxfluxmulticolor,zxfluxmulticolor
45458,zxseries,zxseries
45459,zygarde,zygarde


In [51]:
nodename = "nodeslist_k" + str(k) + ".csv"
nodes_csv.to_csv(nodename, index=None)
nodes_csv

,Id,Label
0,00s,00s
1,00sfashion,00sfashion
2,00sle9dcembre,00sle9dcembre
3,06jul,06jul
4,072100thermalinsulation,072100thermalinsulation
...,...,...
45456,zxflux,zxflux
45457,zxfluxmulticolor,zxfluxmulticolor
45458,zxseries,zxseries
45459,zygarde,zygarde
